In [5]:
from datasets import load_dataset

In [6]:
dataset = load_dataset('HuggingFaceTB/cosmopedia-100k')

In [6]:
df = dataset["train"].to_pandas()

In [17]:
import pandas as pd

df = pd.read_csv("data/cosmopedia-with-embeddings.csv")

In [18]:
df.head()

,Unnamed: 0,prompt,text_token_length,text,seed_data,format,audience,embedding
0,0,"Here is an extract from a webpage: ""What can c...",585,When you've been involved in an auto accident...,web_samples_v2,blogpost,general,"[-0.04932722449302673, 0.005096479784697294, 0..."
1,1,"Here is an extract from a webpage: ""The LISA P...",896,Course Unit: LISA Pathfinder Mission and Grav...,web_samples_v1,textbook_academic_tone,college_students,"[-0.07238882780075073, -0.0395820327103138, 0...."
2,2,"Here is an extract from a webpage: ""Recording ...",701,Title: Making Mathematics Accessible: The Imp...,web_samples_v1,blogpost,general,"[-0.05812391638755798, -0.002327580004930496, ..."
3,3,"Here is an extract from a webpage: ""DoP Jules ...",808,The Performing Arts encompass many different ...,web_samples_v2,blogpost,general,"[-0.021054651588201523, -0.04698284715414047, ..."
4,4,Write an educational story (3-5 paragraphs) ta...,497,"It was a bright, sunny day and Maria was exci...",openhermes2.5,story_children,young_children,"[0.01722874864935875, 0.016011737287044525, 0...."


In [19]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("thenlper/gte-small")

In [ ]:
embeddings = embedding_model.encode(df["text"], show_progress_bar=True, batch_size=32)

In [21]:
import ast
import numpy as np

embeddings = df["embedding"].to_numpy()
embeddings = np.array([ast.literal_eval(item) for item in embeddings])
embeddings

array([[-0.04932722,  0.00509648,  0.04378261, ..., -0.03130245,
         0.05591073,  0.00125157],
       [-0.07238883, -0.03958203,  0.0269948 , ..., -0.01371068,
        -0.00131882,  0.01340186],
       [-0.05812392, -0.00232758,  0.03529689, ...,  0.02320929,
         0.02175455,  0.04959507],
       ...,
       [-0.02138558,  0.01342722,  0.05929201, ..., -0.04456983,
         0.01026244,  0.02929242],
       [-0.0046029 ,  0.00226186,  0.05602154, ..., -0.05386383,
         0.01121266, -0.02191613],
       [-0.03295866, -0.01070711,  0.02379923, ..., -0.02127721,
         0.09072022,  0.04464527]], shape=(100000, 384))

In [22]:
from umap import UMAP
umap_model = UMAP(n_components=5, n_neighbors=5, min_dist=0.2, metric='cosine', random_state=42)
# reduced_embeddings = umap_model.fit_transform(embeddings)

In [23]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=50, metric="euclidean", cluster_selection_method="eom")

In [24]:
from bertopic import BERTopic
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, hdbscan_model=hdbscan_model)
topics, probs = topic_model.fit_transform(df["text"], embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
TOKENIZERS_PARALLELISMTo disable this warning, you can either:
=(true | false)
To disable this warning, you can either:
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 	- Avoid using `

In [25]:
reduced_embeddings = UMAP(n_components=2, n_neighbors=5, metric='cosine', random_state=42).fit_transform(embeddings)

In [ ]:
topic_model.visualize_documents(df["text"], reduced_embeddings=reduced_embeddings, hide_annotations=True)